Day 21 (https://adventofcode.com/2023/day/21)

In [1]:
import numpy as np

with open('./inputs/day21.txt', 'r') as f:
    garden_map = np.array([list(l.strip()) for l in f.readlines()])

    # I'll brute force part 1, knowing full well that probs won't work for part 2
    possibilities = [tuple(np.argwhere(garden_map=='S')[0])]
    for i in range(64):
        new_possibilities = []
        for p in possibilities:
            if p[0] > 0 and garden_map[(p[0]-1, p[1])] != '#':
                new_possibilities.append((p[0]-1, p[1]))
            if p[1] > 0 and garden_map[(p[0], p[1]-1)] != '#':
                new_possibilities.append((p[0], p[1]-1))
            if p[0]+1 < garden_map.shape[0] and garden_map[(p[0]+1, p[1])] != '#':
                new_possibilities.append((p[0]+1, p[1]))
            if p[1]+1 < garden_map.shape[1] and garden_map[(p[0], p[1]+1)] != '#':
                new_possibilities.append((p[0], p[1]+1))

        possibilities = list(set(new_possibilities))

    print('Answer to Day 21, Part 1:', len(possibilities))

    # very difficult
    # lot's of manual and pen-on-paper stuff that I don't feel like showing here,
    # but I got the answer
    part2_ans = 7424*(202300**2) + \
        7388*(202301**2) + \
        (933 + 941 + 948 + 956)*202300 - \
        (893 + 907 + 916 + 913)*202301

    print('Answer to Day 21, Part 2:', part2_ans)

Answer to Day 21, Part 1: 3646
Answer to Day 21, Part 2: 606188414811259


Day 22 (https://adventofcode.com/2023/day/22)

In [2]:
import numpy as np

with open('./inputs/day22.txt', 'r') as f:
    bricks = []
    for line in f.readlines():
        split_line = line.strip().split('~')
        bricks.append(
            {
                'x1': int(split_line[0].split(',')[0]),
                'y1': int(split_line[0].split(',')[1]),
                'z1': int(split_line[0].split(',')[2]),
                'x2': int(split_line[1].split(',')[0]),
                'y2': int(split_line[1].split(',')[1]),
                'z2': int(split_line[1].split(',')[2])
            }
        )

    brick_map = np.zeros(shape=(
        max(max(b['x1'], b['x2']) for b in bricks)+1,
        max(max(b['y1'], b['y2']) for b in bricks)+1,
        max(max(b['z1'], b['z2']) for b in bricks)+1,
    )) - 1

    for i, b in enumerate(bricks):
        brick_map[
            min(b['x1'], b['x2']) : max(b['x1'], b['x2'])+1,
            min(b['y1'], b['y2']) : max(b['y1'], b['y2'])+1,
            min(b['z1'], b['z2']) : max(b['z1'], b['z2'])+1,
        ] = i

    settled_brick_map = np.zeros(shape=brick_map.shape) - 1

    supported_by_dict = {}
    for i in sorted(range(len(bricks)), key = lambda x: min(bricks[x]['z1'], bricks[x]['z2'])):
        brick_blocks = np.argwhere(brick_map==i)
        underbrick_map = settled_brick_map[
            min(brick_blocks[:, 0]) : max(brick_blocks[:, 0])+1,
            min(brick_blocks[:, 1]) : max(brick_blocks[:, 1])+1,
            0 : min(brick_blocks[:, 2])
        ]
        nonempty_underbricks = np.argwhere(underbrick_map!=-1)
        m = len(nonempty_underbricks) and max(nonempty_underbricks[:, 2])

        supported_by_dict[i] = set(int(underbrick_map[tuple(x)]) for x in nonempty_underbricks if x[2] == m)
        settled_brick_map[
            min(brick_blocks[:, 0]) : max(brick_blocks[:, 0])+1,
            min(brick_blocks[:, 1]) : max(brick_blocks[:, 1])+1,
            m + 1 : max(brick_blocks[:, 2]) - min(brick_blocks[:, 2]) + m + 2
        ] = i

    single_point_of_failure_bricks = set(next(iter(b)) for b in supported_by_dict.values() if len(b)==1)
    print('Answer to Day 22, Part 1:', len(bricks) - len(single_point_of_failure_bricks))

    supporting_dict = {k : set(x for x in supported_by_dict if k in supported_by_dict[x]) for k in supported_by_dict}

    def get_num_falling(disintegrate_index):
        falling_bricks = [disintegrate_index]
        keep_going = True
        while keep_going:
            keep_going = False
            for falling_brick in falling_bricks:
                for potential_fall in supporting_dict[falling_brick]:
                    if potential_fall not in falling_bricks and all(b in falling_bricks for b in supported_by_dict[potential_fall]):
                        keep_going = True
                        falling_bricks.append(potential_fall)

        return len(falling_bricks) - 1
    
    print('Answer to Day 22, Part 2:', sum(get_num_falling(b) for b in single_point_of_failure_bricks))

Answer to Day 22, Part 1: 405
Answer to Day 22, Part 2: 61297


Day 23 (https://adventofcode.com/2023/day/23)

In [3]:
import numpy as np
import networkx as nx

with open('./inputs/day23.txt', 'r') as f:
    hike_map = np.array([list(l.strip()) for l in f.readlines()])
    hike_graph = nx.grid_2d_graph(n=hike_map.shape[0], m=hike_map.shape[1], create_using=nx.DiGraph)
    hike_graph.remove_nodes_from(map(tuple, np.argwhere(hike_map=='#')))

    for i, j in np.argwhere(hike_map=='v'):
        for x, y in hike_graph[(i, j)].copy():
            if i + 1 != x:
                hike_graph.remove_edge((i, j), (x, y))

    for i, j in np.argwhere(hike_map=='^'):
        for x, y in hike_graph[(i, j)].copy():
            if i - 1 != x:
                hike_graph.remove_edge((i, j), (x, y))

    for i, j in np.argwhere(hike_map=='>'):
        for x, y in hike_graph[(i, j)].copy():
            if j + 1 != y:
                hike_graph.remove_edge((i, j), (x, y))

    for i, j in np.argwhere(hike_map=='<'):
        for x, y in hike_graph[(i, j)].copy():
            if j - 1 != y:
                hike_graph.remove_edge((i, j), (x, y))

    simple_paths = list(nx.all_simple_paths(hike_graph, min(hike_graph),  max(hike_graph)))
    simple_paths.sort(key=len, reverse=True)

    print('Answer to Day 23, Part 1:', len(simple_paths[0])-1)

    simplified_graph = nx.grid_2d_graph(n=hike_map.shape[0], m=hike_map.shape[1], create_using=nx.Graph)
    simplified_graph.remove_nodes_from(map(tuple, np.argwhere(hike_map=='#')))
    for node in filter(lambda x: len(simplified_graph[x])==2, hike_graph):
        neighbors = list(simplified_graph.neighbors(node))
        simplified_graph.add_edge(
            neighbors[0],
            neighbors[1],
            weight=simplified_graph[node][neighbors[0]].get('weight', 1) + simplified_graph[node][neighbors[1]].get('weight', 1)
        )
        simplified_graph.remove_node(node)

    def longest_simple_path(graph, source, target, path_history=[], path_cost=0):
        path_history = path_history.copy()
        path_history.append(source)
        if source==target:
            return path_history, path_cost

        possible_paths = []
        for n in graph[source]:
            if n not in path_history:
                possible_paths.append(
                    longest_simple_path(
                        graph,
                        source=n,
                        target=target,
                        path_history=path_history,
                        path_cost=path_cost+graph[source][n]['weight']
                    )
                )

        possible_paths = list(filter(lambda x: x is not None, possible_paths))
        if len(possible_paths) > 0:
            return sorted(possible_paths, key=lambda x: x[1], reverse=True)[0]
        else:
            return None
        
    path, cost = longest_simple_path(simplified_graph, source=min(simplified_graph), target=max(simplified_graph))
    print('Answer to Day 23, Part 2:', cost)

Answer to Day 23, Part 1: 2386
Answer to Day 23, Part 2: 6246
